In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool
from functools import partial
import time
import json

import subprocess
from subprocess import PIPE, Popen
import os
from tqdm import tqdm

word2type = {}
def worker(entity_name, i):
    global word2type
    entity_name = entity_name
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    query2 = '''
    SELECT ?object 
    WHERE {dbr:''' + entity_name.replace('(','\(').replace(')','\)').replace('*','\*').replace(',','\,') + '''  rdf:type  ?object }'''

    sparql.setQuery(query2)  # the previous query as a literal string
    sparql.setReturnFormat(JSON)

    tmp = sparql.query()
    one_res = tmp.convert()

    #if one_res['results']['bindings'] == []:

    for result in one_res['results']['bindings']:
        if 'http://dbpedia.org/ontology/' in  result['object']['value']: 
            print(result['object']['value'].split('/')[-1]) 
            return entity_name, result['object']['value'].split('/')[-1]


def collectMyResult(result):
    global word2type
    # if result:
    try:
        word2type[result[0]] = result[1]
            # with open("./word2type.json","w",encoding='utf-8') as f:
            #     json.dump(word2type,f)
        print("Got result {}".format(result[0]))
    except Exception as e:
        print('error:', e)
def abortable_worker(func, *args, **kwargs):
    timeout = 30
    p = ThreadPool(1)
    res = p.apply_async(func, args=args)
    try:
        out = res.get(timeout)  # Wait timeout seconds for func to complete.
        return out
    except multiprocessing.TimeoutError:
        print("Aborting due to timeout")
        raise

if __name__ == "__main__":
    word2type = {}
    with open("./need_type.json","r",encoding='utf-8') as f: # entities need type
            need_type = json.load(f)
    tic = time.time()
    pool = multiprocessing.Pool(processes=24,maxtasksperchild=1)

    featureClass = [[need_type[i], i] for i in tqdm(range(0,len(need_type),1))] #list of arguments
    for f in featureClass:
        abortable_func = partial(abortable_worker, worker, timeout=30)
        pool.apply_async(abortable_func, args=f,callback=collectMyResult)
    pool.close()
    pool.join()
    
    toc = time.time()
    shijian = toc-tic
    print("time consuming:",shijian)

    with open("./word2type.json","w",encoding='utf-8') as f:
            json.dump(word2type,f)
    print(len(word2type))